In [ ]:
from nbdev import *
%nbdev_default_export b1259
%load_ext autoreload
%autoreload 2
!date

Cells will be exported to light_curves.b1259,
unless a different module is specified after an export flag: `%nbdev_export special.module`
Fri May  7 16:30:11 PDT 2021


# B1259-63 Analysis
> Create a B1259-63 (aka PSR J1302-6350) light curve using Bayesian Blocks  

* [HESS high-energy results](https://arxiv.org/pdf/astro-ph/0506280.pdf)

* [Previouis *fermi* paper](https://arxiv.org/pdf/1912.05868.pdf)

In [ ]:
%nbdev_export
import numpy as np
import matplotlib.pyplot as plt
from wtlike  import Config, PointSource, WtLike, UTC, MJD

from utilities.ipynb_docgen import *

In [ ]:
%nbdev_export

class B1259(WtLike):
    
    tp, period = 55544.694, 1236.7243
    
    def __init__(self, **kwargs):
        time_bins =kwargs.pop('time_bins', (0,0,1))
        super().__init__('PSR B1259-63', time_bins=time_bins, **kwargs)
        self.applyBB(key='B1259_BB')

    @property
    def mjd_dates(self):
        return [self.tp+n*self.period for n in range(4)]
         
    def date_info(self):
        return pd.DataFrame([dict(MJD=round(d,3), UTC=UTC(d)) for d in self.mjd_dates ])
    
    def stacked_plots(self, fignum=3, ylim=(2,200), ts_min=4):
 
        tp, p =self.tp, self.period
        fig, axx = plt.subplots(4,1, sharex=True, sharey=True, figsize=(12,12), num=fignum)
        plt.subplots_adjust(hspace=0.)
        for i, ax in enumerate(axx.flatten()):
            self.plot_BB( ax=ax, tzero=tp+i*p, xlim=(-60,150), ylim=ylim, ts_min=ts_min,
                     colors=('coral', 'bisque', 'blue'),
                     fmt='o', yscale='log', xlabel='',ylabel='', 
                        source_name=self.source.name if i==0 else '')
            ax.text(0.02, 0.05, UTC(tp+i*p)[:4], transform=ax.transAxes)
            ax.axvline(0, color='grey', ls=':')
        axx[-1].set_xlabel('days about periastron', fontsize=20)
        fig.text(0.04, 0.5, 'Relative Flux', va='center', rotation='vertical', fontsize=20)
        fig.width=600
        return fig

In [ ]:
# preload
daily = B1259()


SourceData: photons and exposure for PSR B1259-63: Restoring from cache with key "PSR B1259-63_data"
B1259: Source PSR B1259-63 with:
	 data:     1,335,825 photons from   2008-08-04 to 2021-05-06
	 exposure: 3,351,183 intervals from 2008-08-04 to 2021-05-06
CellData: Bin photon data into 4657 1-day bins from 54683.0 to 59340.0
LightCurve: select 4509 cells for fitting with e>0.5 & n>2
B1259_BB: Restoring from cache
Partitioned 4509 cells into 32 blocks, using LikelihoodFitness 
LightCurve: Loaded 32 / 32 cells for fitting


In [ ]:
#daily.plot(ylim=(0.2, 55000),xlim=(55400, 55000), yscale='log', ts_min=0, ts_bar_min=2);

In [ ]:
%nbdev_collapse_input
def B1259( clear=False):
    r"""
    ## Fit to all data

    {date}
    
    Create a `WtLike` object with all the data
    
    {outp}
 
    Combine the likelihoods for all the data, check the fit. 
    Normalization should be very close to 1.0, a check on the weights.
    {fig1}
    
    Note that the "significant count factor", $\sigma_{{flux}} \times \sqrt{{N}}$, is {sig_cnt_factor:.0f}.
 
    ## The full daily-interval light curve, showing the BB partitions
    {fig2}
    
    ## Expand about each periastron
 
    #### Periastron dates

    Assuming {period}-day orbital period, the MJD and UTC values are:
    
    {utc}
 
    Expand the above about those dates

    {fig3}
    
    ## Recent detail 
    {out4}{fig4}
    
    """
    global daily #  make availlable for follow-up cells

 
    pd.set_option('display.precision', 4)#, 'display.colheader_justify','left')

    
    with capture_print('Output from analysis: create cells, fit each, run BB partition, fit partitions.') as outp:
        if daily is None or clear:
            daily = B1259Periastron(time_bins=(0,0,1), clear=clear)
            daily.applyBB()
        else:
            print('(already done)')
    lc = daily.fits
    bb_lc = daily.bb_fit
    period  = daily.period

    # Fig 1 -- fit to combined data
    
    lka = daily.full_likelihood()
    fig1,ax = plt.subplots(figsize=(4,2))
    lka.plot(ax=ax, xlim=(0.5, 1.5) )
    fi = lka.fit_info(); 
    sig_cnt_factor =  fi['sig_flux']*np.sqrt(fi['counts'])
    
    bb_df = daily.bb_table()
    df_text = monospace(str(bb_df), 'BB fit table', open=True)

    # fig 2: full light curbe
    plt.rc('font', size=16)
    fig2 = figure( daily.plot_BB(fignum=2), 
                  width=600)
    
    # fig 3 -- stack light curves
    utc = monospace(str(daily.date_info())) 
    fig3 = figure( daily.stacked_plots( ylim=(4,300), fignum=3), width=600)
    
    # fig 4 -- recent with orbit
    tzero=daily.mjd_dates[-1]
    with capture_print('Recent 1/2-day bins, new BB') as out4:
        half_day = daily.view((tzero+50, 0,1/2))
        half_day.applyBB()
    fig4 = figure(
           half_day.plot_BB(fignum=4, yscale='log', colors=('coral', 'lightgrey', 'blue'),
                 tzero=tzero, xlim=(50, 90 ), xlabel='Days after 2021 Periastron',
                         ylim=(2,400), figsize=(12,9)), 
        width=600)

   
    return locals()

nbdoc(B1259, clear=False)

## Fit to all data

2021-05-07 17:31

Create a `WtLike` object with all the data

<details  class="nbdoc-description" >  <summary> Output from analysis: create cells, fit each, run BB partition, fit partitions. </summary>  <div style="margin-left: 5%"><pre>(already done)<br></pre></div> </details>

Combine the likelihoods for all the data, check the fit. 
Normalization should be very close to 1.0, a check on the weights.
<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 1">  <a href="images/B1259_fig_01.png" title="images/B1259_fig_01.png">    <img src="images/B1259_fig_01.png" alt="Figure 1 at images/B1259_fig_01.png" >   </a> </figure>
</div>


Note that the "significant count factor", $\sigma_{flux} \times \sqrt{N}$, is 174.

## The full daily-interval light curve, showing the BB partitions
<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 2">  <a href="images/B1259_fig_02.png" title="images/B1259_fig_02.png">    <img src="images/B1259_fig_02.png" alt="Figure 2 at images/B1259_fig_02.png" width=600>   </a> </figure>
</div>


## Expand about each periastron

#### Periastron dates

Assuming 1236.7243-day orbital period, the MJD and UTC values are:

<div style="margin-left: 5%"><pre>         MJD               UTC<br>0  55544.694  2010-12-14 16:39<br>1  56781.418  2014-05-04 10:02<br>2  58018.143  2017-09-22 03:25<br>3  59254.867  2021-02-09 20:48</pre></div>

Expand the above about those dates

<div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 3">  <a href="images/B1259_fig_03.png" title="images/B1259_fig_03.png">    <img src="images/B1259_fig_03.png" alt="Figure 3 at images/B1259_fig_03.png" width=600>   </a> </figure>
</div>


## Recent detail 
<details  class="nbdoc-description" >  <summary> Recent 1/2-day bins, new BB </summary>  <div style="margin-left: 5%"><pre>CellData: Bin photon data into 70 12-hour bins from 59304.9 to 59339.9<br>LightCurve: select 70 cells for fitting with e&gt;0.5 & n&gt;2<br>Partitioned 70 cells into 6 blocks, using LikelihoodFitness <br>LightCurve: Loaded 6 / 6 cells for fitting<br></pre></div> </details><div class="nbdoc_image">
<figure style="margin-left: 5%" title="Figure 4">  <a href="images/B1259_fig_04.png" title="images/B1259_fig_04.png">    <img src="images/B1259_fig_04.png" alt="Figure 4 at images/B1259_fig_04.png" width=600>   </a> </figure>
</div>



## Questions for Discussion, with 2/28 Scargle comments

#### How do I interpret the apparent abrupt transitions? 
>Think of BB as the best step-function representation
of the data.  It does not mean that the discontinuities 
are claimed to be real -- the underlying light curve
is most likely continuous and smooth at some level.  
BB is just saying that there is a statistically significant
change from one block to the next.

Compare with a [flare from 3C 279](https://tburnett.github.io/wtlike/bayesian.html#bb_overplot), which has measurements rather than limits for each day and the time scale is in the day range. 

#### Is the day interval appropriate? 
>BB can always benefit from finer sampling, so if you
have data on shorter intervals (or even photon data)
you can get more information out ... e.g. a better 
determination of the time of "transitions" (see above).



#### I know that there is a ~4% systematic for the individual daily measurements, seen in the Geminga data. Does this matter?
>Systematic errors are a problem ... BB is not magic here.

## My observations, plans

There are two related questions here. 
1. what is the actual behavior of B1259, besides the obvious flares? Can we characterize each periastron to detect changes? Kent Wood is very interested in this.
2. How well does BB work, when applied to daily likelihoods, many of which are just limits, but do contain information? This is a question that Jeff Scargle had never considered before, but is quite interested in.


#### Run Simulations

I'll use the [wtlike simulation capability](https://tburnett.github.io/wtlike/simulation) to test the sensitivity to various light curves, when daily fits are all or mostly limits.

#### Add Systematics to Likelihood
The presence of a systematic error in the exposure--I've measured 4% with Geminga, slightly invalidates the likelihood. A fix is to widen it, which I can easily do using the [wtlike Poisson representation](https://tburnett.github.io/wtlike/poisson#Poisson). 

# Comparison with Tyrel's gtlike results

In [ ]:
%nbdev_collapse_input
remote_file = '/nfs/farm/g/glast/u/tyrelj/forToby/FluxLC_output_PSRB1259.fits'
local_copy = '/home/burnett/fromTyrel/FluxLC_output_PSRB1259.fits'
from astropy.io import fits

with fits.open(local_copy) as hdus: 
    tdata = hdus[1].data

flux_factor = 4.28e7 # derived before, so apply here
tdf = pd.DataFrame(dict(t=MJD((tdata.Start+tdata.Stop)/2).round(1),
                        TS = tdata.TS.astype(float).round(1),
                        flux = (tdata.flux.astype(float)*flux_factor).round(1),
                        flux_error = (tdata.flux_error.astype(float)*flux_factor).round(1),
                        limit = (tdata.UpperLimit.astype(float)*flux_factor).round(1),
                       )
                    ); 
#tdf.describe(percentiles=[])

#### Get my results for these dates

%nbdev_collapse_input
lc=daily.lc; 
mdf = lc.dataframe.query('59319>t>59093').copy()
mdf.loc[:,'ts']= mdf.fit.apply(lambda x: x.ts)
mdf.loc[:,'flux']=mdf.fit.apply(lambda x: round(x.flux,1) )
mdf.loc[:,'errors']= mdf.fit.apply(lambda x: (np.array(x.errors)-x.flux).round(1))
mdf.loc[:,'limit']= mdf.fit.apply(lambda x: x.limit.round(1))
#mdf['t ts flux errors limit'.split()].describe(percentiles=[])

### Use the last one, at 59318.5, +63, to make tentative normalization


f = mdf.iloc[-1].fit
print(f'Compare point at {mdf.iloc[-1].t}: TS: me {f.ts:.1f}, Tyrel {tdf.iloc[-1].TS:.1f}')
a,b = f.flux, tdf.iloc[-1].flux; a,b, 
print(f'flux ratio check: {a/b:.3e}')

fig, (ax1,ax2,ax3) = plt.subplots(1,3,figsize=(15,5))
ax1.plot(mdf.ts, tdf.TS, 'o');
ax1.set(xlabel='wtlike TS', ylabel='gtlike TS')
ax1.grid(alpha=0.5)
mtscut= (mdf.ts>4).values
ax2.plot(mdf.flux, tdf.flux, 'o');
ax2.plot(mdf.flux[mtscut], tdf[mtscut].flux, 'or', label='wtlike TS>4');
ax2.plot([0,70], [0,70], '--', color='grey');
ax2.set(xlabel='wtlike flux', ylabel=f'gtlike flux x {flux_factor:.2e}', xlim=(0,70), ylim=(0,70))
ax2.legend();ax2.grid(alpha=0.5);

ttscut=(tdf.TS>4).values
ax3.plot(mdf.flux, tdf.flux, 'o');
ax3.plot(mdf.flux[ttscut], tdf[ttscut].flux, 'or', label='gtlike TS>4');
ax3.plot([0,70], [0,70], '--', color='grey');
ax3.legend()
ax3.set(xlabel='wtlike flux', ylabel=f'gtlike flux x {flux_factor:.2e}', xlim=(0,70), ylim=(0,70))
ax3.legend();ax3.grid(alpha=0.5);

### Add my TS, flux to Tyrel list

In [ ]:
%nbdev_collapse_input
tdf.loc[:,'wt_ts']= mdf.ts.values
tdf.loc[:,'wt_flux']=mdf.flux.values
tdf.loc[:,'wt_errors'] = mdf.errors.values

pa = daily; tzero = pa.date_info().MJD[3]
fig, ax = plt.subplots(figsize=(15,4))
mcut=mdf.ts.values>4
tcut=tdf.TS.values>4
ax.plot((mdf.t-tzero)[mcut], mdf.flux[mcut], 'x',ms=10, label='wtlike')
ax.plot((tdf.t-tzero)[tcut], tdf.flux[tcut], '+', ms=15, label='gtlike')
ax.set(yscale='linear', ylim=(1,90), xlim=(-60, 70), xlabel='Days about periastron', 
       ylabel='Relative Flux');
ax.legend(loc='upper center', title='TS>4 cut on respective data sets')
ax.grid(alpha=0.5);

## Generate table of 3-day fits, to compare with LightCurveRepository

In [ ]:
wt3 = daily.view((0., 0, 3))

wt3.plot_BB();

ft = wt3.flux_table()

ft.to_csv('../b1259_3day.csv')